In [9]:
from importlib import reload

import task1_work
reload(task1_work)
from task1_work import *

import task2_work
reload(task2_work)
from task2_work import *

import countries as ct

path1="data/title.akas.tsv.gz"
path2="data/title.basics.tsv.gz"
path3="data/title.ratings.tsv.gz"

# Results:

In [4]:
df_analysis, end = prepareDataset2(path1, path2, path3,
                              verbose = True)

Loading data... 

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


Done!
Preprocessing... Done!


In [10]:
ha = hegemonyAnalysis(df_analysis,
                      end = datetime.now().year,
                      K = 10000, 
                      verbose = True)

Computing cinematic impacts... Done!
Computing hegemony dataframes... Analyzing gdp...
Analyzing population...
Analyzing gdp pc...
Done!


In [8]:
ha

{'GDP':                            weak impact  strong impact
 Brazil                               7              6
 Canada                             -16            -12
 China                              -23            -29
 Croatia                             44             74
 Denmark                             11             19
 Finland                             31             35
 France                              -4            -18
 Germany                            -16            -17
 Hong Kong                           22             16
 Hungary                             45             47
 Iceland                             90             74
 India                                2              3
 Indonesia                           11            -12
 Iran, Islamic Republic of           29             31
 Israel                               0             17
 Italy                              -21             -9
 Japan                               -4             -5
 Me

# Thinking:

In [2]:
df, df3 = load_dataframes(path1, path2, path3)
df_analysis = prepareDataset(df, df3)

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [3]:
K = 10000
df_analysis = df_analysis.loc[df_analysis["numVotes"] > K]

rank_weak = df_analysis.groupby(by="region")["numVotes"].mean().astype(int)
rank_weak.index = [ct.countries[x] for x in rank_weak.index]
rank_weak = list(rank_weak.sort_values(ascending=False).index)
rank_weak

['United States',
 'Brazil',
 'India',
 'United Arab Emirates',
 'South Korea',
 'Lithuania',
 'Portugal',
 'Indonesia',
 'Iran, Islamic Republic of',
 'Japan',
 'Norway',
 'Slovakia',
 'Hungary',
 'France',
 'South Africa',
 'Timor-Leste',
 'Finland',
 'Latvia',
 'Iceland',
 'Romania',
 'Spain',
 'Poland',
 'Cyprus',
 'Hong Kong',
 'Germany',
 'Sweden',
 'Sierra Leone',
 'Mexico',
 'Turkey',
 'United Kingdom',
 'Philippines',
 'Afghanistan',
 'Denmark',
 'China',
 'Somalia',
 'Canada',
 'Croatia',
 'Italy',
 'Israel',
 'Netherlands',
 'Ethiopia']

In [4]:
df_analysis["score"] = df_analysis["averageRating"] * df_analysis["numVotes"]
rank_strong = df_analysis.groupby(by="region")["score"].sum()/df_analysis.groupby(by="region")["numVotes"].sum()
rank_strong.index = [ct.countries[x] for x in rank_strong.index]
rank_strong = list(rank_strong.sort_values(ascending=False).index)
rank_strong

['Lithuania',
 'India',
 'Brazil',
 'Croatia',
 'United Arab Emirates',
 'Iran, Islamic Republic of',
 'Romania',
 'Poland',
 'Turkey',
 'Hungary',
 'Sierra Leone',
 'Japan',
 'Israel',
 'Timor-Leste',
 'Finland',
 'Portugal',
 'Norway',
 'Sweden',
 'South Korea',
 'Denmark',
 'Germany',
 'Italy',
 'Mexico',
 'Philippines',
 'Afghanistan',
 'South Africa',
 'Canada',
 'Indonesia',
 'Hong Kong',
 'Netherlands',
 'Somalia',
 'France',
 'Cyprus',
 'United Kingdom',
 'Spain',
 'United States',
 'Iceland',
 'Slovakia',
 'Latvia',
 'China',
 'Ethiopia']

# Collective analysis

In [11]:
country_list = rank_strong.copy()
country_list.extend(rank_weak)
country_list = list(set(country_list))

rank_dict = {}
for indic in ['GDP', 'Population', 'GDP PC']:
    print(f"Analyzing {indic.lower()}...")
    rank_dict[indic] = {}
    rank = WorldBankRank(indic)
    for country in country_list:
        try:
            impact_weak = rank.index(country) - rank_weak.index(country)
            impact_strong = rank.index(country) - rank_strong.index(country)
        except ValueError:
            print(f"Country '{country}' not found in {indic.lower()} World Bank Database.")
            impact_weak = pd.NA
            impact_strong = pd.NA
            
        rank_dict[indic][country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    rank_dict[indic] = pd.DataFrame.from_dict(rank_dict[indic]).T.dropna().sort_index()

Analyzing gdp...
Country 'Afghanistan' not found in gdp World Bank Database.
Analyzing population...
Analyzing gdp pc...
Country 'Afghanistan' not found in gdp pc World Bank Database.


In [12]:
rank_dict['GDP']

,weak impact,strong impact
Brazil,7,6
Canada,-26,-17
China,-32,-38
Croatia,38,71
Cyprus,79,69
Denmark,2,15
Ethiopia,16,16
Finland,30,32
France,-7,-25
Germany,-22,-18


# Analysis for GDP

In [64]:
rank_dict = {}
gdp_rank = WorldBankRank("GDP")
for country in country_list:
    try:
        impact_weak = gdp_rank.index(country) - rank_weak.index(country)
        impact_strong = gdp_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest GDP World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in World Bank Database.
Country 'Afghanistan' not found in World Bank Database.


# Analysis for population

In [76]:
rank_dict = {}
for country in country_list:
    try:
        impact_weak = pop_rank.index(country) - rank_weak.index(country)
        impact_strong = pop_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest population World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in latest population World Bank Database.


,weak impact,strong impact
Afghanistan,2,9
Brazil,5,4
Canada,0,8
China,-34,-40
Croatia,91,125
Cyprus,135,121
Denmark,78,91
Ecuador,25,56
Ethiopia,-32,-32
Finland,99,99


# Analysis for GDP P.C.

In [77]:
rank_dict = {}
for country in country_list:
    try:
        impact_weak = gdpPC_rank.index(country) - rank_weak.index(country)
        impact_strong = gdpPC_rank.index(country) - rank_strong.index(country)
    except ValueError:
        print(f"Country '{country}' not found in latest GDP P.C. World Bank Database.")
        impact_weak = pd.NA
        impact_strong = pd.NA
        
    rank_dict[country] = {'weak impact': impact_weak, 'strong impact': impact_strong}
    
rank_df = pd.DataFrame.from_dict(rank_dict).T.dropna().sort_index()
rank_df

Country 'Virgin Islands, U.S.' not found in latest GDP P.C. World Bank Database.
Country 'Afghanistan' not found in latest GDP P.C. World Bank Database.


,weak impact,strong impact
Brazil,78,77
Canada,-21,-13
China,35,29
Croatia,16,50
Cyprus,8,-6
Denmark,-27,-14
Ecuador,59,90
Ethiopia,121,121
Finland,-3,-3
France,10,-9
